In [9]:
# adaptation of this code https://github.com/docongminh/Classification-SIFT-SVM
# to use AKAZE instead of SIFT
import cv2
import os
import glob
import sklearn
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split, GridSearchCV

In [10]:
# reading of input pictures


def imread_img(path_image):

    return cv2.imread(path_image)


def label_image(path_to_data):

    list_label = os.listdir(path_to_data)

    label2id = {}
    for id_, value in enumerate(list_label):
        label2id[value] = id_

    data_train = []
    data_label = []

    for label in list_label:
        for img in glob.glob(path_to_data + "/" + label + "/*"):
            image = imread_img(img)
            label_image = label2id[label]

            data_train.append(image)
            data_label.append(label_image)

    return data_train, data_label, label2id


# TEST
# data_train, data_label, label2id = label_image('casting_512x512/')
# print(data_train[0])
# cv2.imwrite('test.jpg', data_train[0])
# print(data_label[0])
# print(label2id)

In [11]:
# AKAZE and Bag of words


def extract_akaze_features(list_image):

    image_descriptors = []
    akaze = cv2.AKAZE_create()
    for image in list_image:
        _, descriptor = akaze.detectAndCompute(image, None)
        image_descriptors.append(descriptor)

    return image_descriptors


def kmean_bow(all_descriptors, num_cluster):
    bow_dict = []

    kmeans = KMeans(n_clusters=num_cluster)
    kmeans.fit(all_descriptors)

    bow_dict = kmeans.cluster_centers_

    return bow_dict


def create_feature_bow(image_descriptors, BoW, num_cluster):

    X_features = []

    for i in range(len(image_descriptors)):
        features = np.array([0] * num_cluster)

        if image_descriptors[i] is not None:
            distance = cdist(image_descriptors[i], BoW)

            argmin = np.argmin(distance, axis=1)

            for j in argmin:
                features[j] += 1
        X_features.append(features)

    return X_features

In [16]:
# read files, compute features, fit SVM model with Cross Val

path = "casting_512x512"
print(path)
data_train, label, label2id = label_image(path)
print("path ok")

image_descriptors = extract_akaze_features(data_train)
print("descriptors ok")

all_descriptors = []
for descriptor in image_descriptors:
    if descriptor is not None:
        for des in descriptor:
            all_descriptors.append(des)

num_cluster = 60
BoW = kmean_bow(all_descriptors, num_cluster)
print("bow ok")

X_features = create_feature_bow(image_descriptors, BoW, num_cluster)
print("features ok")

X_train, X_test, Y_train, Y_test = train_test_split(
    X_features, label, test_size=0.2, random_state=1
)
model_svm = sklearn.svm.SVC(C=30, random_state=0)
parameters = [{"C": [20, 25, 30, 35, 40, 45]}]
grid_model = GridSearchCV(
    estimator=model_svm, param_grid=parameters, scoring="f1", cv=10
)
print("start fit")
grid_model.fit(X_train, Y_train)
print("fit ok")

print("best score: ", grid_model.best_score_)
print("best_params: ", grid_model.best_params_)

casting_512x512
path ok
descriptors ok
bow ok
features ok
start fit
fit ok
best score:  0.9988235294117647
best_params:  {'C': 25}
